# Application: Cavity Flow

One of the most common validation cases in CFD is the lid-driven cavity flow.  We take a square cavity and set the velocity of the lid to some constant value.  The flow within the cavity is driven by the lid, a spiral pattern forms and two distinctive pressure zones are visible in the upper corners against the lid.

<img src='./figures/drivencavity.svg' width=350>

In [ ]:
import numpy

The poisson equation is an elliptic PDE which almost always means using an iterative solver.  We're going to use the Jacobi method.  There are better ways, but that's beside the point.  

\begin{align}
p_{i,j}^{n+1} &= \frac{1}{4}\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}+p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) \\
&-\frac{\rho \Delta x}{16} \left( \frac{2}{\Delta t} \left(u_{i+1,j} - u_{i-1,j} + v_{i,j+1} - v_{i,j-1}\right) \right . \\
&-\frac{2}{\Delta x}\left(u_{i,j+1} - u_{i,j-1} \right) \left(v_{i+1,j} - v_{i-1,j} \right) \\
&- \left . \frac{\left(u_{i+1,j} - u_{i-1,j} \right)^2}{\Delta x} 
- \frac{ \left(v_{i,j+1} - v_{i,j-1} \right)^2 }{\Delta x} \right) \\
\end{align}

In [ ]:
def velocity_term(b, rho, dt, u, v, dx):
    b[1:-1, 1:-1] = (
        rho * dx / 16 * 
        (2 / dt * (u[2:, 1:-1] - 
                    u[:-2, 1:-1] + 
                    v[1:-1, 2:] - 
                    v[1:-1, :-2]) - 
        2 / dx * (u[1:-1, 2:] - u[1:-1, :-2]) *
                 (v[2:, 1:-1] - v[:-2, 1:-1]) - 
        (u[2:, 1:-1] - u[:-2, 1:-1])**2 / dx - 
        (v[1:-1, 2:] - v[1:-1, :-2])**2 / dx)
                     )

    return b

In [ ]:
def pressure_poisson(p, b, l2_target):
    iter_diff = l2_target + 1
    n = 0
    while iter_diff > l2_target and n <= 500:

        pn = p.copy()
        p[1:-1,1:-1] = (.25 * (pn[2:, 1:-1] +
                               pn[:-2, 1:-1] +
                               pn[1:-1, 2:] +
                               pn[1:-1, :-2]) -
                               b[1:-1, 1:-1])

        p[:, 0] = p[:, 1]   #dp/dy = 0 at y = 0
        p[:, -1] = 0        #p = 0 at y = 2
        p[0, :] = p[1, :]   #dp/dx = 0 at x = 0
        p[-1, :] = p[-2, :] #dp/dy = 0 at x = 2
      
        
        if n % 10 == 0:
            iter_diff = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
                    
        n += 1
        
    return p

In [ ]:
from snippets.ns_helper import cavity_flow

In the interests of brevity, we're only going to worry about the pressure poisson solver.  There rest of the 2D Navier-Stokes solution is encapsulated in the function `cavity_flow`

We'll also load up pickled initial conditions, so we can reliably compare final solutions.

In [ ]:
import pickle

In [ ]:
def run_cavity():
    nx = 41
    ny = 41
    with open('IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dt = .005
    nt = 1000
    
    u, v, p = cavity_flow(u, v, p, nt, dt, dx, 
                         velocity_term, 
                         pressure_poisson, 
                         rtol=1e-4)
    
    return u, v, p

So what does this all do?  Let's check it out.

In [ ]:
u, v, p = run_cavity()

In [ ]:
%matplotlib inline
from snippets.ns_helper import quiver_plot

In [ ]:
quiver_plot(u, v, p)

#### Save NumPy answers for comparison

In [ ]:
with open('numpy_ans.pickle', 'wb') as f:
    pickle.dump((u, v, p), f)

Let's profile the `cavity_flow` function and see if there's a specific place that's really hurting our performance.

In [ ]:
%timeit run_cavity()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f cavity_flow run_cavity()

## Where is the bottleneck?

Clearly the PPE is the problem here, so let's use `numba` to rewrite it.  

In [ ]:
from numba import jit

In [ ]:
# %load snippets/ppe_numba.py

In [ ]:
u_numba, v_numba, p_numba = run_cavity()

In [ ]:
assert numpy.allclose(p, p_numba)
assert numpy.allclose(u, u_numba)
assert numpy.allclose(v, v_numba)

In [ ]:
quiver_plot(u_numba, v_numba, p_numba)

In [ ]:
%timeit run_cavity()

In [ ]:
%lprun -f cavity_flow run_cavity()

## One more bit of optimization?

In [ ]:
@jit(nopython=True)
def velocity_term(b, rho, dt, u, v, dx):
    I, J = b.shape
    
    for i in range(1, I):
        for j in range(1, J):
            b[i, j] = (
            rho * dx / 16 * 
            (2 / dt * (u[i + 1, j] - 
                      u[i - 1, j] + 
                      v[i, j + 1] - 
                      v[i, j - 1]) - 
            2 / dx * (u[i, j + 1] - u[i, j - 1]) * 
                     (v[i + 1, j] - v[i - 1, j]) - 
            (u[i + 1, j] - u[i - 1, j])**2 / dx - 
            (v[i, j + 1] - v[i, j - 1])**2 / dx)
            )
    return b